# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [11]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 14
generator = square_root_generator(limit)
sum_val = 0

for i, sqrt_value in enumerate(generator):
    sum_val += sqrt_value
    print(f'sum for {i + 1} =', sum_val)
    print('--->', sqrt_value)


sum for 1 = 1.0
---> 1.0
sum for 2 = 2.414213562373095
---> 1.4142135623730951
sum for 3 = 4.146264369941973
---> 1.7320508075688772
sum for 4 = 6.146264369941973
---> 2.0
sum for 5 = 8.382332347441762
---> 2.23606797749979
sum for 6 = 10.83182209022494
---> 2.449489742783178
sum for 7 = 13.47757340128953
---> 2.6457513110645907
sum for 8 = 16.30600052603572
---> 2.8284271247461903
sum for 9 = 19.30600052603572
---> 3.0
sum for 10 = 22.4682781862041
---> 3.1622776601683795
sum for 11 = 25.7849029765595
---> 3.3166247903554
sum for 12 = 29.249004591697254
---> 3.4641016151377544
sum for 13 = 32.854555867161245
---> 3.605551275463989
sum for 14 = 36.59621325393519
---> 3.7416573867739413


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [16]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


In [13]:
%%capture
!pip install dlt[duckdb] # Install dlt with all the necessary DuckDB dependencies

In [21]:
import dlt

# define the connection to load to.
# We now use duckdb, but you can switch to Bigquery later
generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='generators')

In [22]:
# we can load any generator to a table at the pipeline destnation as follows:
info = generators_pipeline.run(people_1(),
										table_name="people_1",
										write_disposition="replace")

# the outcome metadata is returned by the load and we can inspect it by printing it.
print(info)

Pipeline dlt_colab_kernel_launcher load step completed in 0.20 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1707693693.1942582 is LOADED and contains no failed jobs


In [25]:
# show outcome

import duckdb

conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

# and the data

print("\n\n\n http_download table below:")

p_1 = conn.sql("SELECT * FROM people_1").df()
display(p_1)

# As you can see, the same data was loaded in both cases.

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people_1            │
│ people_2            │
└─────────────────────┘




 http_download table below:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707693693.1942582,BduY4Pt1T+s3Xg,None
1,2,Person_2,27,City_A,1707693693.1942582,zqqgfemOq+bDEA,None
2,3,Person_3,28,City_A,1707693693.1942582,ukOU/To69LPmtQ,None
3,4,Person_4,29,City_A,1707693693.1942582,jLd9MmJEwfXxNw,None
4,5,Person_5,30,City_A,1707693693.1942582,Sjh5KWobOmr/Uw,None
5,3,Person_3,33,City_B,1707693702.115469,G4kauqpT4A1OIQ,Job_3
6,4,Person_4,34,City_B,1707693702.115469,0riGN1EtTSEflw,Job_4
7,5,Person_5,35,City_B,1707693702.115469,PMrO9mrqOrykKw,Job_5
8,6,Person_6,36,City_B,1707693702.115469,jXnixA82I9ZoPQ,Job_6
9,7,Person_7,37,City_B,1707693702.115469,DCIze9y4g1+rqw,Job_7





 stream_download table below:


,id,name,age,city,occupation,_dlt_load_id,_dlt_id
0,3,Person_3,33,City_B,Job_3,1707693073.9396498,YQlXsOLf5ukZ5A
1,4,Person_4,34,City_B,Job_4,1707693073.9396498,hbaE59DR2bl/UQ
2,5,Person_5,35,City_B,Job_5,1707693073.9396498,kOJv8uYe/86vcg
3,6,Person_6,36,City_B,Job_6,1707693073.9396498,/NZ5QfdKniabsw
4,7,Person_7,37,City_B,Job_7,1707693073.9396498,LdYL37/dPbQVcA
5,8,Person_8,38,City_B,Job_8,1707693073.9396498,cXhgUg5+GqSzmQ


In [26]:
p_1 = conn.sql("SELECT SUM(age) FROM people_1").df()
display(p_1)

,sum(age)
0,353.0


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [29]:
# define the connection to load to.
# We now use duckdb, but you can switch to Bigquery later
pipeline = dlt.pipeline(destination='duckdb', dataset_name='generators')



# run with merge write disposition.
# This is so scaffolding is created for the next example,
# where we look at merging data

info = pipeline.run(people_1(),
										table_name="people_1_pk",
										write_disposition="replace",
                    primary_key="id")

print(info)

info = pipeline.run(people_2(),
										table_name="people_1_pk",
										write_disposition="merge",
                    primary_key="id")

print(info)

Pipeline dlt_colab_kernel_launcher load step completed in 0.26 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1707694343.2697854 is LOADED and contains no failed jobs
Pipeline dlt_colab_kernel_launcher load step completed in 0.43 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1707694344.0114884 is LOADED and contains no failed jobs


In [31]:
p_1 = conn.sql("SELECT * FROM people_1_pk").df()
display(p_1)

p_1 = conn.sql("SELECT SUM(age) FROM people_1_pk").df()
display(p_1)

,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707694343.2697854,N+nnVpLmDTEGIg,None
1,2,Person_2,27,City_A,1707694343.2697854,Of7aklntNY8vGw,None
2,8,Person_8,38,City_B,1707694344.0114884,z29TC3VoVm8Q1A,Job_8
3,4,Person_4,34,City_B,1707694344.0114884,8OPqFkoqdyJFyw,Job_4
4,5,Person_5,35,City_B,1707694344.0114884,xeE3Vck7bLTyGw,Job_5
5,7,Person_7,37,City_B,1707694344.0114884,iwukoDiXfU7NiA,Job_7
6,3,Person_3,33,City_B,1707694344.0114884,xQ98ICt52QzuOQ,Job_3
7,6,Person_6,36,City_B,1707694344.0114884,cN3VI25I/gMUWw,Job_6


,sum(age)
0,266.0


Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX